<a href="https://colab.research.google.com/github/davofas/randomization-/blob/main/Project_Assignments/RandomizationAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation Assignment



In module 1, I proposed the problem of having 10 cases of water with 10 bottles in each case and 10 oz in each bottle.  One case is known to be filled with one ounce less than all the other bottles.  A former student proposed a random selection method for finding the case with the bottles that have less water.  Essentially you'll pull one bottle and test it.  If it is 9 ounces, you have found the case that is short.  Let's explore this random test.

1. Write a random function call that returns 10 ounces 9 out of 10 times and 9 ounces one out of ten.

  
2. What is the expected weight of a random pull?

3. Test your expected weight by running a simulation drawing at least 100 bottles and taking the average of the wieghts.  Does it agree with your theoretical result?

4. We are particuarly interested in how many pulls would be required to find the case that was short.  If you test one at a time, how many pulls do you expect before you find the case that is short?

5. Create a function that creates a sequence of pulling bottles and stops when you find the bottle with 9 ounces.  Test this function a bunch of times.  Do you notice anything odd when comparing it to your previous result?

6. Propose a modification to the coding you did in the first step that would improve the results.  Hint:  Consider a different data structure that would be more applicable to the real world case.

In [24]:
import random

def pull_bottle():
    # Return 9 oz with 10% probability, 10 oz with 90% probability
    if random.random() < 0.1:
        return 9
    else:
        return 10




In [16]:
#2
expected_weight = 0.1 * 9 + 0.9 * 10
print(f"Theoretical expected weight: {expected_weight} oz")


Theoretical expected weight: 9.9 oz


In [25]:
##3
random.seed(42)

total_weight = 0
num_draws = 100

for _ in range(num_draws):
    total_weight += pull_bottle()

average_weight = total_weight / num_draws
print(f"Average weight from {num_draws} draws: {average_weight:.2f} oz")



Average weight from 100 draws: 9.87 oz


In [29]:
def pull_bottle():
    return random.choices([10, 9], weights=[9, 1])[0]

In [30]:
def find_short_case():
    pulls = 0
    while True:
        pulls += 1
        if pull_bottle() == 9:
            return pulls

random.seed(42)
num_simulations = 1000
total_pulls = 0

for _ in range(num_simulations):
    total_pulls += find_short_case()

average_pulls = total_pulls / num_simulations
print(f"Average pulls to find short case over {num_simulations} simulations: {average_pulls:.2f}")


Average pulls to find short case over 1000 simulations: 10.18


the average number of pulls is ~10, but since there are only 10 cases in total, it's possible and happens in simulations to require more than 10 pulls to find the short case . This is "odd" because intuitively, with only 10 cases, you might think you'd find it in at most 10 pulls if checking cases systematically here, pulls are random with replacement, so you could keep drawing from the 9 " good" cases repeatedly before hitting the bad one.

## Simulating Plates

In my household, there is a stack of plates that are differing in colors; pink, blue, black and green.  Of the 24 plates, 6 are each color.  My daughter will not eat dinner if it is not served on a pink plate.  

1.  Compute the probability that you draw a pink plate.
2. Compute the probability that of the four plates drawn for family dinner, atleast one of them is pink.
3. Create a randomization and test the previous two questions answers through randomization.  Discuss the results.
4. Let's make this a bit more complicated and indicative of how we live.  The plates are in a stack where the last one in, is the first one out.  Create a data structure that can account for this.
5. Also, we normally wash the plates when about 12 (sometimes more, sometimes less) collect in the dishwasher.  At that point, they are shuffled before returning to the stack in the cabinet.  Add this to your randomization and compute the probability of pulling exactly one and at least one pink plate when setting the table for four.
6. Since my daughter will not eat unless on the pink plate, most of the pink plates have migrated to the top of the stack.  This happened through a rejection of pull process.  In this, if we did not have at least one pink, more plates were removed from the stack until a pink was found.  Those rejected plates were then returned to the stack.  Create a function for doing this.
7. We have lived in the house for 1000 days.  Run your experiment on your stack always appeasing my daughter with at least one pink plate.  How often do you have all pink plates in the stack?
8. Explore what your final stack looks like by creating a usage count for each of the plates.  Have all the pink plates made it to the top and are some not in regular use?  

Probability of Drawing a Pink Plate:
Total plates: 24, with 6 pink.
P(Pink) = 6 / 24 = 1/4 = 0.25

In [31]:
prob_at_least_one_pink = 1 - (18/24)*(17/23)*(16/22)*(15/21)
print(f"Probability of at least one pink in 4 draws: {prob_at_least_one_pink:.3f}")

Probability of at least one pink in 4 draws: 0.712


In [32]:

import random

# Plates setup
plates = ['pink']*6 + ['blue']*6 + ['black']*6 + ['green']*6

# Function to draw n plates
def draw_plates(n, plate_list):
    random.shuffle(plate_list)
    return plate_list[:n]

random.seed(42)
num_sims = 10000

# Simulate P(Pink in single draw)
p_pink_sim = sum(draw_plates(1, plates.copy())[0] == 'pink' for _ in range(num_sims)) / num_sims
print(f"Simulated P(Pink): {p_pink_sim:.4f} (Theoretical: 0.2500)")

# Simulate P(At least one pink in 4 draws)
p_at_least_sim = sum('pink' in draw_plates(4, plates.copy()) for _ in range(num_sims)) / num_sims
print(f"Simulated P(At least one pink in 4): {p_at_least_sim:.4f} (Theoretical: 0.6836)")

Simulated P(Pink): 0.2505 (Theoretical: 0.2500)
Simulated P(At least one pink in 4): 0.7144 (Theoretical: 0.6836)


the simulated results are very close to the theoretical probabilities.
while Random sampling may also introduces small variations, but with 10,000 trials, the results pretty much confirm the theory.

In [33]:
#4
import random

# Plates setup
plates = ['pink']*6 + ['blue']*6 + ['black']*6 + ['green']*6

# Function to draw plates from a stack with dishwasher handling
def draw_from_stack(n, stack, dishwasher):
    drawn = []
    for _ in range(n):
        if not stack:
            random.shuffle(dishwasher)
            stack.extend(dishwasher)
            dishwasher.clear()
        plate = stack.pop()
        drawn.append(plate)
        dishwasher.append(plate)
        # Wash/shuffle if dishwasher full
        if len(dishwasher) >= 12:
            random.shuffle(dishwasher)
            stack.extend(dishwasher)
            dishwasher.clear()
    return drawn

# Simulation
random.seed(42)
num_sims = 10000
exactly_one = 0
at_least_one = 0

for _ in range(num_sims):
    stack = plates.copy()
    random.shuffle(stack)
    dishwasher = []
    draw = draw_from_stack(4, stack, dishwasher)
    pink_count = draw.count('pink')
    if pink_count == 1:
        exactly_one += 1
    if pink_count >= 1:
        at_least_one += 1

print(f"P(Exactly one pink in 4 with stack/dishwasher): {exactly_one/num_sims:.4f}")
print(f"P(At least one pink in 4 with stack/dishwasher): {at_least_one/num_sims:.4f}")

P(Exactly one pink in 4 with stack/dishwasher): 0.4623
P(At least one pink in 4 with stack/dishwasher): 0.7153


In [43]:
#5
def rejection_draw(stack, dishwasher):
    drawn = []

    # Draw 4 plates initially
    for _ in range(4):
        if not stack:
            random.shuffle(dishwasher)
            stack.extend(dishwasher)
            dishwasher.clear()
        plate = stack.pop()
        drawn.append(plate)
        dishwasher.append(plate)

    # If no pink in initial 4, keep drawing until we get one
    if 'pink' not in drawn:
        extra_draws = []
        while True:
            if not stack:
                random.shuffle(dishwasher)
                stack.extend(dishwasher)
                dishwasher.clear()
            plate = stack.pop()
            drawn.append(plate)
            dishwasher.append(plate)
            if plate == 'pink':
                break
            extra_draws.append(plate)

        # Return rejected non-pinks to top of stack
        stack.extend(reversed(extra_draws))

    # Wash dishwasher if full
    if len(dishwasher) >= 12:
        random.shuffle(dishwasher)
        stack.extend(dishwasher)
        dishwasher.clear()

    return drawn[:4]


In [47]:
import random
from collections import Counter

random.seed(42)
num_days = 1000
all_pink_days = 0
usage_counter = Counter()

for _ in range(num_days):
    stack = plates.copy()
    random.shuffle(stack)
    dishwasher = []
    drawn_plates = rejection_draw(stack, dishwasher)
    usage_counter.update(drawn_plates)

    if stack.count('pink') == 6:
        all_pink_days += 1

print(f"Days with all pink plates in stack: {all_pink_days} out of {num_days}")
print("Plate usage counts (approximate over 1000 days):")
print(usage_counter)

Days with all pink plates in stack: 14 out of 1000
Plate usage counts (approximate over 1000 days):
Counter({'green': 1024, 'pink': 1015, 'blue': 996, 'black': 965})
